## --- Day 5: Hydrothermal Venture ---

You come across a field of hydrothermal vents on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.

They tend to form in lines; the submarine helpfully produces a list of nearby lines of vents (your puzzle input) for you to review. For example:
```
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
```
Each line of vents is given as a line segment in the format x1,y1 -> x2,y2 where x1,y1 are the coordinates of one end the line segment and x2,y2 are the coordinates of the other end. These line segments include the points at both ends. In other words:

    An entry like 1,1 -> 1,3 covers points 1,1, 1,2, and 1,3.
    An entry like 9,7 -> 7,7 covers points 9,7, 8,7, and 7,7.

For now, only consider horizontal and vertical lines: lines where either x1 = x2 or y1 = y2.

So, the horizontal and vertical lines from the above list would produce the following diagram:
```
.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....
```
In this diagram, the top left corner is 0,0 and the bottom right corner is 9,9. Each position is shown as the number of lines which cover that point or . if no line covers that point. The top-left pair of 1s, for example, comes from 2,2 -> 2,1; the very bottom row is formed by the overlapping lines 0,9 -> 5,9 and 0,9 -> 2,9.

To avoid the most dangerous areas, you need to determine the number of points where at least two lines overlap. In the above example, this is anywhere in the diagram with a 2 or larger - a total of 5 points.

Consider only horizontal and vertical lines. At how many points do at least two lines overlap?

In [1]:
class PointMap:
  def __init__(self, filename, read_angled=False):
    self._points = {}
    self._read_angled = read_angled
    self.read_file(filename)

  def add_line(self, line_points):
    (x1, y1), (x2, y2) = line_points
    if x1 == x2: # Vertical line
      for y in range(min(y1, y2), max(y1, y2)+1):
        self.add_point(x1, y)
    elif y1 == y2: # Horizontal line
      for x in range(min(x1, x2), max(x1, x2)+1):
        self.add_point(x, y1)
    elif self._read_angled: # 45-deg line (slope 1 or -1)
      if x1 > x2:
        # Always add points L->R
        x1, x2 = x2, x1
        y1, y2 = y2, y1

      slope = 1 if y2 > y1 else -1

      for i in range(x2 - x1 + 1):
        self.add_point(x1+i, y1+(i*slope))

  def add_point(self, x, y):
    point = (x, y)
    if point in self._points:
      self._points[point] += 1
    else:
      self._points[point] = 1

  def filter_points(self, min_lines):
    return {point: num for point, num in self._points.items() if num >= min_lines}

  def read_file(self, filename):
    with open(filename) as f:
      for line in f.readlines():
        p1, p2 = line.rstrip().split(' -> ')
        x1, y1 = p1.split(',')
        x2, y2 = p2.split(',')
        self.add_line([(int(x1), int(y1)), (int(x2), int(y2))])

In [2]:
example1_map = PointMap("./inputs/Day5ex.txt")
example1_solution = 5

assert example1_solution == len(example1_map.filter_points(2))

In [3]:
problem1_map = PointMap("./inputs/Day5.txt")
len(problem1_map.filter_points(2))

8350

### --- Part Two ---

Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider diagonal lines.

Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:

    An entry like 1,1 -> 3,3 covers points 1,1, 2,2, and 3,3.
    An entry like 9,7 -> 7,9 covers points 9,7, 8,8, and 7,9.

Considering all lines from the above example would now produce the following diagram:
```
1.1....11.
.111...2..
..2.1.111.
...1.2.2..
.112313211
...1.2....
..1...1...
.1.....1..
1.......1.
222111....
```
You still need to determine the number of points where at least two lines overlap. In the above example, this is still anywhere in the diagram with a 2 or larger - now a total of 12 points.

Consider all of the lines. At how many points do at least two lines overlap?

In [4]:
x1, y1 = 9,7
x2, y2 = 7,9
points = []

if x1 > x2:
  # Always build L->R
  x1, x2 = x2, x1
  y1, y2 = y2, y1

slope = 1 if y2 > y1 else -1

for i in range(x2 - x1 + 1):
  points.append((x1+i, y1+(i*slope)))

points


[(7, 9), (8, 8), (9, 7)]

In [5]:
example2_map = PointMap("./inputs/Day5ex.txt", read_angled=True)
example2_solution = 12

assert example2_solution == len(example2_map.filter_points(2))

In [6]:
problem2_map = PointMap('./inputs/Day5.txt', read_angled=True)

len(problem2_map.filter_points(2))

19374